In [1]:
%matplotlib inline
import numpy as np
import pandas as pd
import scipy
import sklearn
import spacy
import matplotlib.pyplot as plt
import seaborn as sns
import re
from nltk.corpus import gutenberg, stopwords
from collections import Counter

Supervised NLP requires a pre-labelled dataset for training and testing, and is generally interested in categorizing text in various ways. In this case, we are going to try to predict whether a sentence comes from _Alice in Wonderland_ by Lewis Carroll or _Persuasion_ by Jane Austen. We can use any of the supervised models we've covered previously, as long as they allow categorical outcomes. In this case, we'll try Random Forests, SVM, and KNN.

Our feature-generation approach will be something called _BoW_, or _Bag of Words_. BoW is quite simple: For each sentence, we count how many times each word appears. We will then use those counts as features.  

In [2]:
def perf_measure(y, pred_y):
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y, pred_y)
    FP = cm.sum(axis=0) - np.diag(cm)  
    FN = cm.sum(axis=1) - np.diag(cm)
    TP = np.diag(cm)
    TN = cm[:].sum() - (FP + FN + TP)

    TPR = TP/(TP+FN)
    print('Sensitivity, hit rate, recall, or true positive rate: ', TPR[1])
    TNR = TN/(TN+FP) 
    print('Specificity or true negative rate: ', TNR[1])
    PPV = TP/(TP+FP)
    print('Precision or positive predictive value: ', PPV[1])
    NPV = TN/(TN+FN)
    print('Negative predictive value: ', NPV[1])
    FPR = FP/(FP+TN)
    print('Fall out or false positive rate: ', FPR[1])
    FNR = FN/(TP+FN)
    print('False negative rate: ', FNR[1])
    FDR = FP/(TP+FP)
    print('False discovery rate: ', FDR)
    ACC = (TP+TN)/(TP+FP+FN+TN)
    print('Overall accuracy: ', ACC)

    
def print_confusion_matrix(trained_model, X, y):
    pred_y = trained_model.predict(X)
    print('\nConfusion matrix for test set:')
    print(pd.crosstab(pred_y, y))
    
    print(perf_measure(y, pred_y))

    
def run_train_test_split(model_instance, X, y, pretrained_model=False):
    from sklearn.model_selection import train_test_split
    X_train, X_test, y_train, y_test = train_test_split(X, 
                                                        y,
                                                        test_size=0.4,
                                                        random_state=0)
    if not pretrained_model:
        model_instance.fit(X_train, y_train)
    print('Training set score:', model_instance.score(X_train, y_train))
    print('\nTest set score:', model_instance.score(X_test, y_test))
    print_confusion_matrix(model_instance, X_test, y_test)

    
def run_model(model_instance, X=None, y=None, df=None, outcome_col=None, folds=5, pretrained_model=False):
    from sklearn.model_selection import cross_val_score
    X = X if X.any() else df.drop(outcome_col, axis='columns')
    y = y if y.any() else df[outcome_col]
    
    top_outcome = y.describe().top
    percent_of_top_outcome = round(100 * y.describe().freq / len(y), 2)
    print(f'Class balance: {top_outcome} comprises {percent_of_top_outcome}% of dataset.')
    
    if not pretrained_model:  # we can run cross-validation (which trains the model)
        cv_scores = cross_val_score(model_instance, X, y, cv=folds)
        print(f'\nCross Val ({folds} folds):')
        print(f'Avg: {round(100*cv_scores.mean(), 2)} \nStd: {round(100*cv_scores.std(), 2)}')
        
    run_train_test_split(model_instance, X, y, pretrained_model=pretrained_model)
    print('__DONE__')
    return model_instance, X, y


def get_feature_importance(trained_model, X):
    importances = trained_model.feature_importances_
    indices = np.argsort(importances)[::-1]
    names = [X.columns[i] for i in indices]
    
    # Plot the feature importances of the trained_model
    plt.figure()
    plt.title("Feature importances")
    plt.bar(range(X.shape[1]), importances[indices],
           color="r", align="center")
    plt.xticks(range(X.shape[1]), names)
    plt.xlim([-1, X.shape[1]])
    for tick in plt.gca().get_xticklabels():
        tick.set_rotation(85)
    plt.show()

# Data Prep

# STEP 1: Load text, clean

# NOTE: Can TO 10K CHARS FOR DEV

In [3]:
MAX_CHARACTERS = int(1e6)

In [4]:
def text_cleaner(text):
    # Visual inspection identifies a form of punctuation spaCy does not
    # recognize: the double dash '--'.  Better get rid of it now!
    text = re.sub(r'--',' ',text)
    text = re.sub("[\[].*?[\]]", "", text)
    text = ' '.join(text.split())
    return text

def load_and_clean_text(name, removal_regex):
    text = gutenberg.raw(name)
    text = re.sub(removal_regex, '', text)
    return text_cleaner(text)[0:MAX_CHARACTERS]  # Too big text breaks spaCy with a warming/error

# STEP 2: Load via spacy, prase into sentences (used in df)

In [5]:
def parse_into_doc_and_sents(text, label):
    '''Parse the cleaned novel. This can take a bit.'''
    nlp = spacy.load('en')
    text_doc = nlp(text)
    print('nlp parsing done.')
    text_sents = [[sent, label] for sent in text_doc.sents]
    print('Sentences parsed.')
    return text_doc, text_sents

Time to bag some words!  Since spaCy has already tokenized and labelled our data, we can move directly to recording how often various words occur.  We will exclude stopwords and punctuation.  In addition, in an attempt to keep our feature space from exploding, we will work with lemmas (root words) rather than the raw text terms, and we'll only use the 2000 most common words for each text.

In [6]:
COMMON_WORD_COUNT = 200

In [7]:
# Utility function to create a list of the 2000 most common words.
def bag_of_words(text):
    
    # Filter out punctuation and stop words.
    allwords = [token.lemma_
                for token in text
                if not token.is_punct
                and not token.is_stop]
    
    # Return the most common words.
    return [item[0] for item in Counter(allwords).most_common(COMMON_WORD_COUNT)]
    

# Creates a data frame with features for each word in our common word set.
# Each value is the count of the times the word appears in each sentence.
def bow_features(sentences, common_words):
    
    # Scaffold the data frame and initialize counts to zero.
    df = pd.DataFrame(columns=common_words)
    df['text_sentence'] = sentences[0]
    df['text_source'] = sentences[1]
    df.loc[:, common_words] = 0
    
    # Process each row, counting the occurrence of words in each sentence.
    for i, sentence in enumerate(df['text_sentence']):
        
        # Convert the sentence to lemmas, then filter out punctuation,
        # stop words, and uncommon words.
        words = [token.lemma_
                 for token in sentence
                 if (
                     not token.is_punct
                     and not token.is_stop
                     and token.lemma_ in common_words
                 )]
        
        # Populate the row with word counts.
        for word in words:
            df.loc[i, word] += 1
        
        # This counter is just to make sure the kernel didn't hang.
        if i % 100 == 0:
            print("Processing row {}".format(i))
            
    return df

# STEP 2: turn most common words into features

In [8]:
def create_common_word_features_df(text1_doc, tex2_doc, text1_sents, text2_sents):
    common_words = set(bag_of_words(text1_doc) + bag_of_words(tex2_doc))
    sentences = pd.DataFrame(text1_sents + text2_sents)
    return bow_features(sentences, common_words), common_words

# Run BoW on *Alice* and *Persuasion*

In [9]:
# Group into sentences.
alice_doc, alice_sents = parse_into_doc_and_sents(load_and_clean_text('carroll-alice.txt', r'CHAPTER .*'), 'Caroll')
persuasion_doc, persuasion_sents = parse_into_doc_and_sents(load_and_clean_text('austen-persuasion.txt', r'Chapter \d+'), 'Austen')

nlp parsing done.
Sentences parsed.
nlp parsing done.
Sentences parsed.


In [10]:
# Create our data frame with features. This can take a while to run.
word_counts, common_words = create_common_word_features_df(alice_doc, persuasion_doc, alice_sents, persuasion_sents)

Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing row 4900
Processing r

In [11]:
def get_x_and_y(word_counts):
    Y = word_counts['text_source']
    X = np.array(word_counts.drop(['text_sentence','text_source'], 1))
    return X, Y

# Logistic Regression

In [12]:
X, y = get_x_and_y(word_counts)
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(solver='lbfgs')
alice_persuasion_lr, X, y = run_model(lr, X=X, y=y, folds=5)

Class balance: Austen comprises 68.62% of dataset.

Cross Val (5 folds):
Avg: 89.11 
Std: 1.9
Training set score: 0.9200626959247649

Test set score: 0.9041353383458647

Confusion matrix for test set:
text_source  Austen  Caroll
row_0                      
Austen         1428     160
Caroll           44     496
Sensitivity, hit rate, recall, or true positive rate:  0.7560975609756098
Specificity or true negative rate:  0.970108695652174
Precision or positive predictive value:  0.9185185185185185
Negative predictive value:  0.8992443324937027
Fall out or false positive rate:  0.029891304347826088
False negative rate:  0.24390243902439024
False discovery rate:  [0.10075567 0.08148148]
Overall accuracy:  [0.90413534 0.90413534]
None
__DONE__


## RF

Now let's give the bag of words features a whirl by trying a random forest.

In [13]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split

rfc = ensemble.RandomForestClassifier(n_estimators=10)
_rfc = run_model(rfc, X=X, y=y, folds=5)

Class balance: Austen comprises 68.62% of dataset.

Cross Val (5 folds):
Avg: 86.63 
Std: 2.56
Training set score: 0.9658307210031348

Test set score: 0.8778195488721805

Confusion matrix for test set:
text_source  Austen  Caroll
row_0                      
Austen         1375     163
Caroll           97     493
Sensitivity, hit rate, recall, or true positive rate:  0.7515243902439024
Specificity or true negative rate:  0.9341032608695652
Precision or positive predictive value:  0.8355932203389831
Negative predictive value:  0.8940182054616385
Fall out or false positive rate:  0.06589673913043478
False negative rate:  0.24847560975609756
False discovery rate:  [0.10598179 0.16440678]
Overall accuracy:  [0.87781955 0.87781955]
None
__DONE__


Logistic regression performs a bit better than the random forest.  

# BoW with Gradient Boosting

And finally, let's see what gradient boosting can do:

In [14]:
clf = ensemble.GradientBoostingClassifier()
run_model(clf, X=X, y=y, folds=5)

Class balance: Austen comprises 68.62% of dataset.

Cross Val (5 folds):
Avg: 85.78 
Std: 2.79
Training set score: 0.8840125391849529

Test set score: 0.8740601503759399

Confusion matrix for test set:
text_source  Austen  Caroll
row_0                      
Austen         1444     240
Caroll           28     416
Sensitivity, hit rate, recall, or true positive rate:  0.6341463414634146
Specificity or true negative rate:  0.9809782608695652
Precision or positive predictive value:  0.9369369369369369
Negative predictive value:  0.8574821852731591
Fall out or false positive rate:  0.019021739130434784
False negative rate:  0.36585365853658536
False discovery rate:  [0.14251781 0.06306306]
Overall accuracy:  [0.87406015 0.87406015]
None
__DONE__


(GradientBoostingClassifier(criterion='friedman_mse', init=None,
               learning_rate=0.1, loss='deviance', max_depth=3,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, n_estimators=100,
               n_iter_no_change=None, presort='auto', random_state=None,
               subsample=1.0, tol=0.0001, validation_fraction=0.1,
               verbose=0, warm_start=False), array([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=object), 0       Caroll
 1       Caroll
 2       Caroll
 3       Caroll
 4       Caroll
 5       Caroll
 6       Caroll
 7       Caroll
 8       Caroll
 9       Caroll
 10      Caroll
 11      Caroll
 12      Caroll
 13      Car

Looks like logistic regression is the winner, but there's room for improvement.

# Same model, new inputs

What if we feed the model a different novel by Jane Austen, like _Emma_?  Will it be able to distinguish Austen from Carroll with the same level of accuracy if we insert a different sample of Austen's writing?

First, we need to process _Emma_ the same way we processed the other data, and combine it with the Alice data:

# Test Emma on Alice-Persuasion trained model

In [15]:
emma_regex = r'(VOLUME \w+)|(CHAPTER \w+)'
emma_doc, emma_sents = parse_into_doc_and_sents(load_and_clean_text('austen-emma.txt', emma_regex), 'Austen')
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Alice and Persuasion.
emma_sentences = pd.DataFrame(emma_sents[0:len(alice_sents)])
emma_bow = bow_features(emma_sentences, common_words)

# Combine the Emma sentence data with ONLY the Alice data (exclude Persuasion)
X_Emma_Alice = np.concatenate((
    X[y[y == 'Caroll'].index],
    emma_bow.drop(['text_sentence','text_source'], 1)
), axis=0)

y_Emma_Alice = pd.concat([y[y == 'Caroll'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])


nlp parsing done.
Sentences parsed.
Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600


In [16]:
emma_alice_lr, _X, _y = run_model(alice_persuasion_lr, X=X_Emma_Alice, y=y_Emma_Alice, pretrained_model=True)

Class balance: Austen comprises 50.0% of dataset.
Training set score: 0.8661338661338661

Test set score: 0.8390718562874252

Confusion matrix for test set:
col_0   Austen  Caroll
row_0                 
Austen     588     156
Caroll      59     533
Sensitivity, hit rate, recall, or true positive rate:  0.7735849056603774
Specificity or true negative rate:  0.9088098918083463
Precision or positive predictive value:  0.9003378378378378
Negative predictive value:  0.7903225806451613
Fall out or false positive rate:  0.09119010819165378
False negative rate:  0.22641509433962265
False discovery rate:  [0.20967742 0.09966216]
Overall accuracy:  [0.83907186 0.83907186]
None
__DONE__


Well look at that!  NLP approaches are generally effective on the same type of material as they were trained on. It looks like this model is actually able to differentiate multiple works by Austen from Alice in Wonderland.  Now the question is whether the model is very good at identifying Austen, or very good at identifying Alice in Wonderland, or both...

# Challenge 0:

Recall that the logistic regression model's best performance on the test set was 93%.  See what you can do to improve performance.  Suggested avenues of investigation include: Other modeling techniques (SVM?), making more features that take advantage of the spaCy information (include grammar, phrases, POS, etc), making sentence-level features (number of words, amount of punctuation), or including contextual information (length of previous and next sentences, words repeated from one sentence to the next, etc), and anything else your heart desires.  Make sure to design your models on the test set, or use cross_validation with multiple folds, and see if you can get accuracy above 90%.  

In [17]:
from sklearn import svm
svm_classifier = svm.SVC(gamma='scale')
alice_persuasion_svm = run_model(svm_classifier, X=X, y=y, folds=5, pretrained_model=False)

Class balance: Austen comprises 68.62% of dataset.

Cross Val (5 folds):
Avg: 85.77 
Std: 2.2
Training set score: 0.8755485893416928

Test set score: 0.8665413533834586

Confusion matrix for test set:
text_source  Austen  Caroll
row_0                      
Austen         1449     261
Caroll           23     395
Sensitivity, hit rate, recall, or true positive rate:  0.6021341463414634
Specificity or true negative rate:  0.984375
Precision or positive predictive value:  0.9449760765550239
Negative predictive value:  0.8473684210526315
Fall out or false positive rate:  0.015625
False negative rate:  0.3978658536585366
False discovery rate:  [0.15263158 0.05502392]
Overall accuracy:  [0.86654135 0.86654135]
None
__DONE__


# Challenge 1:
Find out whether your new model is good at identifying Alice in Wonderland vs any other work, Persuasion vs any other work, or Austen vs any other work.  This will involve pulling a new book from the Project Gutenberg corpus (print(gutenberg.fileids()) for a list) and processing it.

Record your work for each challenge in a notebook and submit it below.

## Load and prep Bible text

In [18]:
bible_doc, bible_sents = parse_into_doc_and_sents(load_and_clean_text('bible-kjv.txt', r'\n?\d+\:\d+'), 'Bible')

nlp parsing done.
Sentences parsed.


## Train new model on Bible and Persuasion

In [19]:
bible_persuasion_df, bible_persuasion_common_words = create_common_word_features_df(persuasion_doc, bible_doc, persuasion_sents, bible_sents)


Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing row 4900
Processing r

In [20]:
X_bible_persuasion, y_bible_persuasion = get_x_and_y(bible_persuasion_df)
bible_persuasion_lr, _X, _y = run_model(lr, X=X_bible_persuasion, y=y_bible_persuasion, pretrained_model=False)


Class balance: Bible comprises 65.17% of dataset.

Cross Val (5 folds):
Avg: 96.97 
Std: 0.51
Training set score: 0.9789976133651551

Test set score: 0.9677881173944166

Confusion matrix for test set:
text_source  Austen  Bible
row_0                     
Austen         1382    109
Bible            26   2674
Sensitivity, hit rate, recall, or true positive rate:  0.9608336327703917
Specificity or true negative rate:  0.9815340909090909
Precision or positive predictive value:  0.9903703703703703
Negative predictive value:  0.9268947015425889
Fall out or false positive rate:  0.018465909090909092
False negative rate:  0.03916636722960833
False discovery rate:  [0.0731053  0.00962963]
Overall accuracy:  [0.96778812 0.96778812]
None
__DONE__


# TEST Bible-Persusasion model with Bible-Emma

In [21]:
# Build a new Bag of Words data frame for Emma word counts.
# We'll use the same common words from Bible and Persuasion.

emma_sentences = pd.DataFrame(emma_sents[0:len(bible_sents)])  # Hack in case emma is shorter
emma_bow = bow_features(emma_sentences, bible_persuasion_common_words)

emma_x_to_concat = emma_bow.drop(['text_sentence','text_source'], 1)
bible_x_to_concat = X_bible_persuasion[y_bible_persuasion[y_bible_persuasion == 'Bible'].index]



# Combine the Emma sentence data with ONLY the Bible data (exclude Persuasion)
X_Emma_Bible = np.concatenate((
    bible_x_to_concat,
    emma_x_to_concat
), axis=0)

y_Emma_Bible = pd.concat([y_bible_persuasion[y_bible_persuasion == 'Bible'],
                         pd.Series(['Austen'] * emma_bow.shape[0])])


Processing row 0
Processing row 100
Processing row 200
Processing row 300
Processing row 400
Processing row 500
Processing row 600
Processing row 700
Processing row 800
Processing row 900
Processing row 1000
Processing row 1100
Processing row 1200
Processing row 1300
Processing row 1400
Processing row 1500
Processing row 1600
Processing row 1700
Processing row 1800
Processing row 1900
Processing row 2000
Processing row 2100
Processing row 2200
Processing row 2300
Processing row 2400
Processing row 2500
Processing row 2600
Processing row 2700
Processing row 2800
Processing row 2900
Processing row 3000
Processing row 3100
Processing row 3200
Processing row 3300
Processing row 3400
Processing row 3500
Processing row 3600
Processing row 3700
Processing row 3800
Processing row 3900
Processing row 4000
Processing row 4100
Processing row 4200
Processing row 4300
Processing row 4400
Processing row 4500
Processing row 4600
Processing row 4700
Processing row 4800
Processing row 4900
Processing r

In [22]:
bible_emma_lr, _X, _y = run_model(bible_persuasion_lr, X=X_Emma_Bible, y=y_Emma_Bible, pretrained_model=True)

Class balance: Austen comprises 50.0% of dataset.
Training set score: 0.9591064453125

Test set score: 0.9635664591724643

Confusion matrix for test set:
col_0   Austen  Bible
row_0                
Austen    2665     73
Bible      126   2598
Sensitivity, hit rate, recall, or true positive rate:  0.9726694122051666
Specificity or true negative rate:  0.9548548907201719
Precision or positive predictive value:  0.9537444933920705
Negative predictive value:  0.9733382030679328
Fall out or false positive rate:  0.04514510927982802
False negative rate:  0.027330587794833397
False discovery rate:  [0.0266618  0.04625551]
Overall accuracy:  [0.96356646 0.96356646]
None
__DONE__


# Eval
## Model trained on Bible-Persuasion still works well on Bible-Emma